# Analysis of IoT-Related CVEs and Their Exploitation by APTs

In [1]:
import os # For traversing and reading folders and files
import json # For reading and extracting data from CVE records
import re # Handle regex patterns
import xml.etree.ElementTree as ET # For reading and extracting data from CWE records
import ast # For safely evaluating strings as Python expressions
import pandas as pd # For data cleaning and analysis
import numpy as np # For advanced calculations
import matplotlib.pyplot as plt # For data visualization

## Data Collection
### Extracting Data from the CVE List
There are precisely $260,896$ records in the CVE list; each is represented by a single JSON file. These files contain all the information (though lots of it is incomplete) that will populate our `CVE_df` dataframe. It was clear after creating the initial script that massive discrepencacies existed between the files' structures. This makes it especially complicated to reliably retrieve desired values, but I managed to create a recursive function that performs various data retrieval processes based on the kind of data type the object holds at every level on its way to pull out the desired info. Based on the CVE List schema, I figured that the most important information—and really, the only kinds of information contained in the files that would be useful to us—would be the CVE ID, the CWE ID, the description, the date the record was published, a categorical and numerical severity score, the vector of attack used in the CVE, and attack complexity.



### Extracting Data from the CWE List
The CWE list contains lots of information that could be pertinant to our analysis that's relatively easy to access. Among these I counted the CWE IDs, names, descriptions, related CWEs, the nature of these relationships, the technological vector (web-based, network, etc.), background details, the phases of development wherein the weakness could be introduced, descriptions about these unfortunate events, the likelihood of these weaknesses, the scope and impact of the weakness's common consequences, detection methods and their descriptions and effectiveness, mitigation strategies, potential vulnerabilities, observed vulnerabilities (with direct references to CVE IDs), and more. Because of the observed vulnerabilities, we can merge together these two dataframes and clean and filter the resultant so that we have a comprehensive dataset to work with. Because there's so much text data in the CWE list, we should have plenty of information to feed into a matural language processing pipeline that can help build a classification model to predict the association of a given CVE with the IoT and it's various vulnerabilities.

I created several helper functions to get us from `A` to `B`. The first, I can just feed the XML file. Next, we have a function that takes an element and takes all of the text data inside of it and it's children nodes and returns it as a long sentence. This kind of column data will be useful when we tokenize and lemmatize the text to more accurately search for our desire IoT-vulnerability-related keywords.

### Saving the Data
Parquet is a file type that streamlines the storage and retrieval of columnar data. I also saved a copy of both dataframes to CSV (a basic type of spreadsheet file). I chose to use the default option `None` for the method's `index` parameter, which saves the index of each record in a special kind of metadata range loop. This means it won't take up the kind of memory it would have if the index was actually saved into the dataframe as a separate attribute, but also provides a way to keep track of the records for the purposes of splitting them up between training, test, and validation sets for an ML algorithm should our work come to that.

<span style='font-weight:600;color:#ff8800;background-color:#666672;border-radius:3px;padding-inline:3px;padding-block:1px;'>Don't run these code cells unless you want to overwrite the saved files!</span>

## Loading the Data
I saved the dataframes resulting from our extraction into secondary storage because pulling data from $260,000$ files takes a very long time (1.5+ hours). Now that we have neatly-packaged CSV files, we'll read them into Pandas so we can inspect their cleanliness, merge them together in interest ways, and construct an comprehensive analysis with descriptive statistics, chi-square independence testing, Spearman's rank correlation, and logistic regression. The CWE list specifically contains many columns whose values are are lists of values. Because Pandas doesn't know how to handle these lists natively, I give the parsing function a converter that saves us a cleaning step. That's what "explosive_cols" is; the columns that contain lists of values that we'll explode into new rows during preprocessing.

In [2]:
cves = pd.read_parquet('../data/CVE_V5/CVE_List.parquet')

cves.head(n=1)

,cve_id,cwe_id,cve_state,date_published,description,severity,severity_score,attack_vector,attack_complexity
0,CVE-1999-0001,None,PUBLISHED,2000-02-04T05:00:00,ip_input.c in BSD-derived TCP/IP implementatio...,None,NaN,None,None


In [3]:
cwes = pd.read_parquet('../data/CWE_V4.15/CWE_List.parquet')

cwes.head(n=1)

,cwe_id,name,description,tech_class,bg_details,rel_ids,nature_of_rels,modes_of_intro_phases,modes_of_intro_descs,likelihood,...,consequence_notes,detection_method,detection_desc,detection_effectiveness,mitigation_phases,mitigation_descs,mitigation_effectiveness,mitigate_notes,observed_vulnerabilities,vulnerability_descs
0,1004,Sensitive Cookie Without 'HttpOnly' Flag,The product uses a cookie to store sensitive i...,Web Based,An HTTP cookie is a small piece of data attrib...,[732],[ChildOf],[Implementation],[],Medium,...,"[If the HttpOnly flag is not set, then sensiti...",[Automated Static Analysis],"[Automated static analysis, commonly referred ...",[High],Implementation,[Leverage the HttpOnly flag when setting a sen...,[High],[While this mitigation is effective for protec...,"[CVE-2022-24045, CVE-2014-3852, CVE-2015-4138]",[Web application for a room automation system ...


## Preprocessing
In order to reliably analyze the data, we need to make sure it's clean. This involves several important steps, namely:
1) Understanding the data at a bird's eye level; e.g. number of rows, summary stats about numerical columns, data types (numbers, text, dates, etc.)
2) Handling missing values, redundant white space, inconsistent formatting, and typos
3) Removing duplicates
4) Converting data types into their respective forms for efficient processing

Before feeding the data into a machine learning model (if we get there), we'll have several additional steps to add on to this process:
1) We'll want to encode our categorical data so that we can use math on it
2) We'll split our data into training, validation, and testing sets to ensure our model generalizes well to unseen data
3) We'll have to handle outliers that can make overfitting our model a dangerous probability
4) We'll scale our data around a common mean of `0` and a standard deviation of `1` so that all features contribute equally to the algorithm's output.

### Bird's Eye View of CVE List
Let's start with the CVE List, then traverse the CWEs, and finally, the full dataset once we've merged them.

In [4]:
cves.info() # Overview of the size of the dataset, its null values in each column, and their datatypes
cves.describe() # View summary stats of numerical attributes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260894 entries, 0 to 260893
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   cve_id             260894 non-null  object 
 1   cwe_id             58852 non-null   object 
 2   cve_state          260894 non-null  object 
 3   date_published     257370 non-null  object 
 4   description        246505 non-null  object 
 5   severity           65617 non-null   object 
 6   severity_score     65742 non-null   float64
 7   attack_vector      52410 non-null   object 
 8   attack_complexity  52431 non-null   object 
dtypes: float64(1), object(8)
memory usage: 17.9+ MB


,severity_score
count,65742.000000
mean,6.631829
std,1.752317
min,0.000000
25%,5.400000
50%,6.500000
75%,7.800000
max,10.000000


In [5]:
# Check for incorrect CVE/CWE ID formats
non_ideal_cves = cves[~cves['cve_id'].str.startswith('CVE-', na=True)]
non_ideal_cwes = cves[~cves['cwe_id'].str.startswith('CWE-', na=True)]

print(f'There are {len(non_ideal_cves)} incorrectly-formatted CVE IDs.')
print(f'There are {len(non_ideal_cwes)} incorrectly-formatted CWE IDs.')

There are 0 incorrectly-formatted CVE IDs.
There are 0 incorrectly-formatted CWE IDs.


We'll deal with correcting these redunant categories and any typos in the ["Standardizing Formatting"](#standardizing-formatting) section below; for now we're just trying to see any anomalies.

In [6]:
cves['severity'].value_counts() # Types of severity

severity
MEDIUM      31632
HIGH        24219
CRITICAL     5646
LOW          4085
NONE           33
medium          1
MODERATE        1
Name: count, dtype: int64

In [5]:
cves['attack_vector'].value_counts() # Types of attack vector

attack_vector
NETWORK             37346
LOCAL               11747
ADJACENT_NETWORK     2659
PHYSICAL              614
ADJACENT               44
Name: count, dtype: int64

In [6]:
cves['attack_complexity'].value_counts() # Levels of attack complexity

attack_complexity
LOW     43947
HIGH     8484
Name: count, dtype: int64

In [7]:
cves['cwe_id'].describe() # View summary stats of CWE IDs

count      58852
unique       664
top       CWE-79
freq        8871
Name: cwe_id, dtype: object

In [8]:
cves['date_published'] # View date formats

0              2000-02-04T05:00:00
1              1999-09-29T04:00:00
2              1999-09-29T04:00:00
3              2000-02-04T05:00:00
4              1999-09-29T04:00:00
                    ...           
260889    2024-08-20T23:31:03.646Z
260890    2024-08-20T23:31:05.010Z
260891    2024-08-21T20:20:26.856Z
260892    2024-08-21T20:20:27.045Z
260893    2024-08-21T20:20:27.239Z
Name: date_published, Length: 260894, dtype: object

In [9]:
cves['description']

0         ip_input.c in BSD-derived TCP/IP implementatio...
1         Buffer overflow in NFS mountd gives root acces...
2         Execute commands as root via buffer overflow i...
3         MIME buffer overflow in email clients, e.g. So...
4         Arbitrary command execution via IMAP buffer ov...
                                ...                        
260889    A vulnerability was found in Genexis Tilgin Ho...
260890    A vulnerability classified as critical has bee...
260891    Inappropriate implementation in WebApp Install...
260892    Inappropriate implementation in Custom Tabs in...
260893    Inappropriate implementation in Extensions in ...
Name: description, Length: 260894, dtype: object

In [10]:
cves['cve_state'].value_counts() # Types of state

cve_state
PUBLISHED    246505
REJECTED      14389
Name: count, dtype: int64

In [11]:
# Do the rejected IDs contain any useful info?
cves[cves['cve_state'] == 'REJECTED'].head() # Rejected CVE IDs

,cve_id,cwe_id,cve_state,date_published,description,severity,severity_score,attack_vector,attack_complexity
19,CVE-1999-0020,None,REJECTED,2000-02-04T05:00:00,None,None,NaN,None,None
109,CVE-1999-0110,None,REJECTED,2000-02-04T05:00:00,None,None,NaN,None,None
186,CVE-1999-0187,None,REJECTED,2000-02-04T05:00:00,None,None,NaN,None,None
281,CVE-1999-0282,None,REJECTED,2000-02-04T05:00:00,None,None,NaN,None,None
334,CVE-1999-0335,None,REJECTED,1999-09-29T04:00:00,None,None,NaN,None,None


In [5]:
print('Does the number of CVEs lacking a description exactly match the number of rejected CVEs?')
len(cves[cves['description'].isna()]) == len(cves[cves['cve_state'] == 'REJECTED'])

Does the number of CVEs lacking a description exactly match the number of rejected CVEs?


True

From the looks of it, every record has an ID, a published date, and a state, though some $14000$ have been rejected, because of this contain no other useful information, and can be dropped entirely from the table. The publish date will be converted to a proper datetime format. Every published record has a description. The severity scores are already the proper numerical data type, but their categorical counterparts contain multiple levels of `MEDIUM` (`medium` and `MODERATE`) which, based on the [NVD-documented categories](https://nvd.nist.gov/vuln-metrics/cvss), will be need to be addressed.

### Dropping Unnecessary Information

In [4]:
# Drop rejected CVEs
cves = cves.drop(cves[cves['cve_state'] == 'REJECTED'].index)
cves['cve_state'].value_counts()

cve_state
PUBLISHED    246505
Name: count, dtype: int64

### Correcting Data Types

In [5]:
# Check data types
def check_type(cols):
    for col in cols:
        dtype = type(col)
        print(f'"{col}" is of type {dtype}.')

check_type(cves.columns)

"cve_id" is of type <class 'str'>.
"cwe_id" is of type <class 'str'>.
"cve_state" is of type <class 'str'>.
"date_published" is of type <class 'str'>.
"description" is of type <class 'str'>.
"severity" is of type <class 'str'>.
"severity_score" is of type <class 'str'>.
"attack_vector" is of type <class 'str'>.
"attack_complexity" is of type <class 'str'>.


In [6]:
# Convert publication date to datetime format
cves['date_published'] = pd.to_datetime(cves['date_published'], format='ISO8601', utc=True)

In [7]:
# Convert objects to text data (string)
obj_cols = cves.select_dtypes(include=['object']).columns
cves[obj_cols] = cves[obj_cols].astype('string')

cves.info()

<class 'pandas.core.frame.DataFrame'>
Index: 246505 entries, 0 to 260893
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   cve_id             246505 non-null  string             
 1   cwe_id             58852 non-null   string             
 2   cve_state          246505 non-null  string             
 3   date_published     245989 non-null  datetime64[ns, UTC]
 4   description        246505 non-null  string             
 5   severity           65617 non-null   string             
 6   severity_score     65742 non-null   float64            
 7   attack_vector      52410 non-null   string             
 8   attack_complexity  52431 non-null   string             
dtypes: datetime64[ns, UTC](1), float64(1), string(7)
memory usage: 18.8 MB


### Standardizing Formatting

In [8]:
# Standardize severity scores
cves['severity'] = cves['severity'].replace(['medium', 'MODERATE'], 'MEDIUM')
cves['severity'].value_counts()

severity
MEDIUM      31634
HIGH        24219
CRITICAL     5646
LOW          4085
NONE           33
Name: count, dtype: Int64

In [9]:
# Check for leading or trailing whitespace
str_cols = cves.select_dtypes(include=['string']).columns

def check_whitespace(df, cols):
    for col in cols:
        trimmable = df[col].str.contains(r'^\s+|\s+$', regex=True).sum()
        print(f"Column '{col}' has {trimmable} trimmable whitespace characters.")

# Remove leading or trailing whitespace
cves[str_cols] = cves[str_cols].apply(lambda x: x.str.strip())

check_whitespace(cves, str_cols)

Column 'cve_id' has 0 trimmable whitespace characters.
Column 'cwe_id' has 0 trimmable whitespace characters.
Column 'cve_state' has 0 trimmable whitespace characters.
Column 'description' has 0 trimmable whitespace characters.
Column 'severity' has 0 trimmable whitespace characters.
Column 'attack_vector' has 0 trimmable whitespace characters.
Column 'attack_complexity' has 0 trimmable whitespace characters.


### Removing Duplicates

In [9]:
# Check for duplicates
duplicates = cves.duplicated().sum()
print(f'There are {duplicates} duplicates in the dataset.')

There are 0 duplicates in the dataset.


### Bird's Eye View of CWE List
We still have to handle null values in the CVE list, but since we'll be populating it with data from the CWE list during the merge, we can apply the same techniques to clean our second dataset. Once they've been merged, we'll filter out or impute the null values depending on what works best for the analysis.

We'll want to figure out the same basic facts about the CWE list that we did for the CVE list.

In [111]:
cwes.info() # Overview of the size of the dataset, its null values in each column, and their datatypes
cwes.head(n=1)


<class 'pandas.core.frame.DataFrame'>
Index: 4153 entries, 0 to 4163
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   cwe_id                    4153 non-null   string
 1   name                      4153 non-null   string
 2   description               4153 non-null   string
 3   tech_class                3885 non-null   string
 4   bg_details                4153 non-null   string
 5   rel_id                    1608 non-null   string
 6   nature_of_rel             1608 non-null   string
 7   mode_of_intro_phase       1375 non-null   string
 8   mode_of_intro_desc        520 non-null    string
 9   likelihood                4153 non-null   string
 10  scope_of_consequence      1224 non-null   string
 11  impact_of_consequence     1224 non-null   string
 12  consequence_notes         652 non-null    string
 13  detection_method          1369 non-null   string
 14  detection_desc            135

,cwe_id,name,description,tech_class,bg_details,rel_id,nature_of_rel,mode_of_intro_phase,mode_of_intro_desc,likelihood,...,consequence_notes,detection_method,detection_desc,detection_effectiveness,mitigation_phase,mitigation_desc,mitigation_effectiveness,mitigate_notes,observed_vulnerability,vulnerability_desc
0,CWE-1004,Sensitive Cookie Without 'HttpOnly' Flag,The product uses a cookie to store sensitive i...,Web Based,An HTTP cookie is a small piece of data attrib...,732,ChildOf,Implementation,<NA>,Medium,...,"If the HttpOnly flag is not set, then sensitiv...",Automated Static Analysis,"Automated static analysis, commonly referred t...",High,Implementation,Leverage the HttpOnly flag when setting a sens...,High,While this mitigation is effective for protect...,CVE-2022-24045,Web application for a room automation system h...


Due to the original data potentially having more than one value for a particular column, I had to pull data dynamically for several attributes into what are technically lists of values. For example, a single CWE ID may have multiple related weaknesses or multiple detection methods. There are groups of these attributes that should correspond to other columns in their group with a 1-to-1 relationship, so that each of a CWE ID's related weaknesses can be mapped to their own respective "nature of relationship" values, or that each of a CWE's detection methods has a detection description and a detection effectiveness, for example. All of this makes cleaning this particular dataset significantly more involved than the CVE list. Dealing with other cleaning steps is made much more complicated without first handling the dataset's architectural issues.

The solution I landed on was to pull in data as a parquet file (which can understand a broader range of datatypes that a simple CSV), create a function that would transform the necessary columns into lists, correct various syntactic anomalies, pad each list so that every list across a given row has the same amount of items, and finally explode these lists across multiple rows. This has preserved the 1-to-1 relationship between items in the lists relative to the items in other lists.

In [10]:
# For converting values into manipulable lists
def safe_lit_eval(value):
    try:
        if isinstance(value, np.ndarray):
            value = value.tolist()
        elif isinstance(value, str):
            value = value.strip()
            if ~value.startswith('[') and ~value.endswith(']') and value:
                return [value]
            if value.startswith('[') and value.endswith(']'):
                return ast.literal_eval(value)
        elif isinstance(value, list):
            return value

    except (ValueError, SyntaxError) as e:
        print(f'Warning: Error parsing value "{value}": {e}')

    # If list is empty, return NaN
    if isinstance(value, list) and not value:
        return [np.nan]
    return value

# For converting values into manipulable lists
def eval_lists(df, cols):
    for col in cols:
        df[col] = df[col].apply(safe_lit_eval)
    return df

# For normalizing list values
def normalize_list_syntax(lst):
    if isinstance(lst, list):
        if not lst: # []
            return [np.nan]
        return [np.nan if item == '' or item is None else str(item).strip() for item in lst]
    return lst

# For normalizing list values
def normalize_lists(df, cols):
    for col in cols:
        df[col] = df[col].apply(normalize_list_syntax)
    return df

# For adding items to each list such that it matches the length of the list with
# the most amount of items in the row
def pad_lists(row, cols):
    # Calculate the number of items in the list within the maximum number of items in a given row
    max_len = max(len(row[col]) if isinstance(row[col], list) else 0 for col in cols)
    # Equalize list length
    for col in cols:
        if isinstance(row[col], list):
            row[col] += [np.nan] * (max_len - len(row[col]))
        else:
            row[col] = [np.nan] * max_len
    return row

# For checking if all list values in a given row have the same length
def check_equal_list_lengths(row, cols):
    lengths = [len(row[col]) for col in cols if isinstance(row[col], list)]
    return len(set(lengths)) == 1

# For spreading list items over new rows
def explode_cols(df, cols):
    return df.explode(cols, ignore_index=True)

# For combining all necessary actions
def process_explosive_cols(df, cols):
    df = eval_lists(df, cols)
    df = normalize_lists(df, cols)
    df = df.apply(lambda row: pad_lists(row, cols), axis=1)

    all_rows_valid = df.apply(lambda row: check_equal_list_lengths(row, cols), axis=1).all()
    if all_rows_valid:
        df = explode_cols(df, cols)
        print('Lists have been exploded successfully.')
    else:
        print('All lists in a row need to have the same length to explode.')
    return df

# Columns to process
explosive_cols = [
    'rel_ids',
    'nature_of_rels',
    'modes_of_intro_phases',
    'modes_of_intro_descs',
    'scope_of_consequences',
    'impact_of_consequences',
    'consequence_notes',
    'detection_method',
    'detection_desc',
    'detection_effectiveness',
    'mitigation_phases',
    'mitigation_descs',
    'mitigation_effectiveness',
    'mitigate_notes',
    'observed_vulnerabilities',
    'vulnerability_descs'
]

# Make a copy for testing purposes
# cwes_test1 = cwes.copy(deep=True)

# Process the explosive columns
cwes = process_explosive_cols(cwes, explosive_cols)

Lists have been exploded successfully.


The goal is now to continue preprocessing the data—looking at data types, checking for duplicates and typos, removing whitespace and normalizing capitalization, renaming columns if necessary, and merging the two data tables based on the `observed_vulnerabilities`/`cve_id` attributes.

### Renaming Attributes To Account For Singular Values

In [11]:
# Rename columns
col_names = {
    'rel_ids': 'rel_id',
    'nature_of_rels': 'nature_of_rel',
    'modes_of_intro_phases': 'mode_of_intro_phase',
    'modes_of_intro_descs': 'mode_of_intro_desc',
    'scope_of_consequences': 'scope_of_consequence',
    'impact_of_consequences': 'impact_of_consequence',
    'detection_methods': 'detection_method',
    'detection_descs': 'detection_desc',
    'mitigation_phases': 'mitigation_phase',
    'mitigation_descs': 'mitigation_desc',
    'observed_vulnerabilities': 'observed_vulnerability',
    'vulnerability_descs': 'vulnerability_desc',
}

cwes = cwes.rename(columns=col_names)

In [15]:
cwes.head(3)

,cwe_id,name,description,tech_class,bg_details,rel_id,nature_of_rel,mode_of_intro_phase,mode_of_intro_desc,likelihood,...,consequence_notes,detection_method,detection_desc,detection_effectiveness,mitigation_phase,mitigation_desc,mitigation_effectiveness,mitigate_notes,observed_vulnerability,vulnerability_desc
0,1004,Sensitive Cookie Without 'HttpOnly' Flag,The product uses a cookie to store sensitive i...,Web Based,An HTTP cookie is a small piece of data attrib...,732,ChildOf,Implementation,NaN,Medium,...,"If the HttpOnly flag is not set, then sensitiv...",Automated Static Analysis,"Automated static analysis, commonly referred t...",High,Implementation,Leverage the HttpOnly flag when setting a sens...,High,While this mitigation is effective for protect...,CVE-2022-24045,Web application for a room automation system h...
1,1004,Sensitive Cookie Without 'HttpOnly' Flag,The product uses a cookie to store sensitive i...,Web Based,An HTTP cookie is a small piece of data attrib...,NaN,NaN,NaN,NaN,Medium,...,If the cookie in question is an authentication...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CVE-2014-3852,CMS written in Python does not include the HTT...
2,1004,Sensitive Cookie Without 'HttpOnly' Flag,The product uses a cookie to store sensitive i...,Web Based,An HTTP cookie is a small piece of data attrib...,NaN,NaN,NaN,NaN,Medium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CVE-2015-4138,Appliance for managing encrypted communication...


### Checking For Typos

In [24]:
# Check for typos
cwes['tech_class'].value_counts()


tech_class
                           2999
Not Technology-Specific     382
ICS/OT                      148
Mobile                      143
Web Based                   106
System on Chip               63
Cloud Computing              55
Name: count, dtype: int64

In [25]:
cwes['nature_of_rel'].value_counts()

nature_of_rels
ChildOf       1308
CanPrecede     141
PeerOf          92
nan             34
CanAlsoBe       27
Requires        13
StartsWith       3
Name: count, dtype: int64

In [26]:
cwes['mode_of_intro_phase'].value_counts()

modes_of_intro_phases
Implementation              745
Architecture and Design     340
nan                         138
Operation                   104
System Configuration          9
Manufacturing                 7
Requirements                  7
Integration                   6
Installation                  6
Build and Compilation         4
Documentation                 3
Policy                        2
Patching and Maintenance      1
Bundling                      1
Distribution                  1
Testing                       1
Name: count, dtype: int64

In [27]:
cwes['likelihood'].value_counts()

likelihood
          2804
High       810
Medium     430
Low        120
Name: count, dtype: int64

In [28]:
cwes['detection_method'].value_counts()

detection_methods
nan                                                       639
Automated Static Analysis                                 257
Architecture or Design Review                              62
Dynamic Analysis with Manual Results Interpretation        52
Manual Static Analysis - Source Code                       46
Automated Static Analysis - Source Code                    44
Manual Analysis                                            44
Automated Static Analysis - Binary or Bytecode             34
Fuzzing                                                    32
Manual Static Analysis - Binary or Bytecode                31
Dynamic Analysis with Automated Results Interpretation     31
Automated Dynamic Analysis                                 27
Black Box                                                  21
Manual Static Analysis                                     13
Automated Analysis                                         11
Manual Dynamic Analysis                             

In [29]:
cwes['detection_effectiveness'].value_counts()

detection_effectiveness
nan              639
High             420
SOAR Partial     150
Moderate          54
Limited            9
Opportunistic      6
Name: count, dtype: int64

In [30]:
cwes['mitigation_phase'].value_counts()

mitigation_phases
Implementation              517
Architecture and Design     254
Testing                      55
System Configuration         47
Operation                    40
Build and Compilation        11
Installation                  9
Integration                   6
Requirements                  5
Documentation                 4
Manufacturing                 4
Policy                        2
Distribution                  2
Patching and Maintenance      1
Name: count, dtype: int64

In [31]:
cwes['mitigation_effectiveness'].value_counts()

mitigation_effectiveness
nan                            296
High                            75
Defense in Depth                50
Moderate                        43
Limited                         37
Discouraged Common Practice      9
Incidental                       4
None                             1
Name: count, dtype: int64

### Checking Data Types

In [12]:
check_type(cwes.columns)

# Explicitly convert "objects" to text
cwes[cwes.columns] = cwes[cwes.columns].astype('string')

"cwe_id" is of type <class 'str'>.
"name" is of type <class 'str'>.
"description" is of type <class 'str'>.
"tech_class" is of type <class 'str'>.
"bg_details" is of type <class 'str'>.
"rel_id" is of type <class 'str'>.
"nature_of_rel" is of type <class 'str'>.
"mode_of_intro_phase" is of type <class 'str'>.
"mode_of_intro_desc" is of type <class 'str'>.
"likelihood" is of type <class 'str'>.
"scope_of_consequence" is of type <class 'str'>.
"impact_of_consequence" is of type <class 'str'>.
"consequence_notes" is of type <class 'str'>.
"detection_method" is of type <class 'str'>.
"detection_desc" is of type <class 'str'>.
"detection_effectiveness" is of type <class 'str'>.
"mitigation_phase" is of type <class 'str'>.
"mitigation_desc" is of type <class 'str'>.
"mitigation_effectiveness" is of type <class 'str'>.
"mitigate_notes" is of type <class 'str'>.
"observed_vulnerability" is of type <class 'str'>.
"vulnerability_desc" is of type <class 'str'>.


Every column is now simple text, making it much easier to trim whitespace from each entry and perform any needed string entry.

In [13]:
# Check for leading or trailing whitespace
str_cols = cwes.select_dtypes(include=['string']).columns

# Remove leading or trailing whitespace
cwes[str_cols] = cwes[str_cols].apply(lambda x: x.str.strip())
check_whitespace(cwes, str_cols)

Column 'cwe_id' has 0 trimmable whitespace characters.
Column 'name' has 0 trimmable whitespace characters.
Column 'description' has 0 trimmable whitespace characters.
Column 'tech_class' has 0 trimmable whitespace characters.
Column 'bg_details' has 0 trimmable whitespace characters.
Column 'rel_id' has 0 trimmable whitespace characters.
Column 'nature_of_rel' has 0 trimmable whitespace characters.
Column 'mode_of_intro_phase' has 0 trimmable whitespace characters.
Column 'mode_of_intro_desc' has 0 trimmable whitespace characters.
Column 'likelihood' has 0 trimmable whitespace characters.
Column 'scope_of_consequence' has 0 trimmable whitespace characters.
Column 'impact_of_consequence' has 0 trimmable whitespace characters.
Column 'consequence_notes' has 0 trimmable whitespace characters.
Column 'detection_method' has 0 trimmable whitespace characters.
Column 'detection_desc' has 0 trimmable whitespace characters.
Column 'detection_effectiveness' has 0 trimmable whitespace characters

### Normalizing Observed Vulnerability IDs

In [14]:
# Standardize null values in text columns
cwes[str_cols] = cwes[str_cols].replace([None, 'NaN', 'nan', '', np.nan], pd.NA)


In [15]:
# Find CVE ID values that don't conform to what's required
non_ideal_cves = cwes[~cwes['observed_vulnerability'].str.startswith('CVE-', na=True)]

print(f"There are {len(non_ideal_cves)} observations in which the observed vulnerability's CVE ID is incorrect.")
non_ideal_cves.head(10)

There are 7 observations in which the observed vulnerability's CVE ID is incorrect.


,cwe_id,name,description,tech_class,bg_details,rel_id,nature_of_rel,mode_of_intro_phase,mode_of_intro_desc,likelihood,...,consequence_notes,detection_method,detection_desc,detection_effectiveness,mitigation_phase,mitigation_desc,mitigation_effectiveness,mitigate_notes,observed_vulnerability,vulnerability_desc
383,1260,Improper Handling of Overlap Between Protected...,"The product allows address regions to overlap,...",<NA>,<NA>,119,CanPrecede,Implementation,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,For all of the programmable memory protection ...,High,<NA>,[REF-1100],processor design flaw allows ring 0 code to ac...
427,1277,Firmware Not Updateable,The product does not provide its users with...,Not Technology-Specific,<NA>,<NA>,<NA>,Architecture and Design,Lack of planning during architecture developme...,<NA>,...,<NA>,Architecture or Design Review,Check the consumer or maintainer documentation...,Moderate,<NA>,Design the device to allow for updating the fi...,<NA>,<NA>,[REF-1095],"A hardware ""smart lock"" has weak key generatio..."
705,1391,Use of Weak Credentials,The product uses weak credentials (such as a d...,ICS/OT,<NA>,1390,ChildOf,Requirements,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,Implementation,<NA>,<NA>,<NA>,[REF-1374],Chain: JavaScript-based cryptocurrency library...
1933,332,Insufficient Entropy in PRNG,"The lack of entropy available for, or used by,...",<NA>,<NA>,331,ChildOf,Architecture and Design,<NA>,Medium,...,If a pseudo-random number generator is using a...,<NA>,<NA>,<NA>,Architecture and Design,Use products or modules that conform to FIPS 1...,<NA>,<NA>,[REF-1374],Chain: JavaScript-based cryptocurrency library...
2167,392,Missing Report of Error Condition,The product encounters an error but does not p...,<NA>,<NA>,755,ChildOf,Implementation,<NA>,<NA>,...,Errors that are not properly reported could pl...,<NA>,<NA>,<NA>,Implementation,<NA>,<NA>,<NA>,[REF-1374],Chain: JavaScript-based cryptocurrency library...
3371,703,Improper Check or Handling of Exceptional Cond...,The product does not properly anticipate or ha...,Not Technology-Specific,<NA>,<NA>,<NA>,Architecture and Design,<NA>,<NA>,...,<NA>,Dynamic Analysis with Manual Results Interpret...,<NA>,High,Implementation,<NA>,<NA>,<NA>,[REF-1374],Chain: JavaScript-based cryptocurrency library...
3438,755,Improper Handling of Exceptional Conditions,The product does not handle or incorrectly han...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Medium,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,[REF-1374],Chain: JavaScript-based cryptocurrency library...


I did some digging around MITRE and the NVD and I couldn't find what CVE's the above `[REF]` values pointed to, so they'll need to be taken out.

In [17]:
cwes = cwes[cwes['observed_vulnerability'].str.startswith('CVE-', na=True)]
cwes.shape

(4157, 22)

In [21]:
cwes.head(10)

,cwe_id,name,description,tech_class,bg_details,rel_id,nature_of_rel,mode_of_intro_phase,mode_of_intro_desc,likelihood,...,consequence_notes,detection_method,detection_desc,detection_effectiveness,mitigation_phase,mitigation_desc,mitigation_effectiveness,mitigate_notes,observed_vulnerability,vulnerability_desc
0,CWE-1004,Sensitive Cookie Without 'HttpOnly' Flag,The product uses a cookie to store sensitive i...,Web Based,An HTTP cookie is a small piece of data attrib...,CWE-732,ChildOf,Implementation,<NA>,Medium,...,"If the HttpOnly flag is not set, then sensitiv...",Automated Static Analysis,"Automated static analysis, commonly referred t...",High,Implementation,Leverage the HttpOnly flag when setting a sens...,High,While this mitigation is effective for protect...,CVE-2022-24045,Web application for a room automation system h...
1,CWE-1004,Sensitive Cookie Without 'HttpOnly' Flag,The product uses a cookie to store sensitive i...,Web Based,An HTTP cookie is a small piece of data attrib...,<NA>,<NA>,<NA>,<NA>,Medium,...,If the cookie in question is an authentication...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,CVE-2014-3852,CMS written in Python does not include the HTT...
2,CWE-1004,Sensitive Cookie Without 'HttpOnly' Flag,The product uses a cookie to store sensitive i...,Web Based,An HTTP cookie is a small piece of data attrib...,<NA>,<NA>,<NA>,<NA>,Medium,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,CVE-2015-4138,Appliance for managing encrypted communication...
3,CWE-1007,Insufficient Visual Distinction of Homoglyphs ...,The product displays information or identifier...,Web Based,<NA>,CWE-451,ChildOf,Architecture and Design,This weakness may occur when characters from v...,Medium,...,An attacker may ultimately redirect a user to ...,Manual Dynamic Analysis,"If utilizing user accounts, attempt to submit ...",Moderate,Implementation,Use a browser that displays Punycode for IDNs ...,<NA>,<NA>,CVE-2013-7236,web forum allows impersonation of users with h...
4,CWE-1007,Insufficient Visual Distinction of Homoglyphs ...,The product displays information or identifier...,Web Based,<NA>,<NA>,<NA>,Implementation,<NA>,Medium,...,<NA>,<NA>,<NA>,<NA>,<NA>,Use an email client that has strict filters an...,<NA>,<NA>,CVE-2012-0584,Improper character restriction in URLs in web ...
5,CWE-1007,Insufficient Visual Distinction of Homoglyphs ...,The product displays information or identifier...,Web Based,<NA>,<NA>,<NA>,<NA>,<NA>,Medium,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,CVE-2009-0652,Incomplete denylist does not include homoglyph...
6,CWE-1007,Insufficient Visual Distinction of Homoglyphs ...,The product displays information or identifier...,Web Based,<NA>,<NA>,<NA>,<NA>,<NA>,Medium,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,CVE-2017-5015,web browser does not convert hyphens to punyco...
7,CWE-1007,Insufficient Visual Distinction of Homoglyphs ...,The product displays information or identifier...,Web Based,<NA>,<NA>,<NA>,<NA>,<NA>,Medium,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,CVE-2005-0233,homoglyph spoofing using punycode in URLs and ...
8,CWE-1007,Insufficient Visual Distinction of Homoglyphs ...,The product displays information or identifier...,Web Based,<NA>,<NA>,<NA>,<NA>,<NA>,Medium,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,CVE-2005-0234,homoglyph spoofing using punycode in URLs and ...
9,CWE-1007,Insufficient Visual Distinction of Homoglyphs ...,The product displays information or identifier...,Web Based,<NA>,<NA>,<NA>,<NA>,<NA>,Medium,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,CVE-2005-0235,homoglyph spoofing using punycode in URLs and ...


### Normalizing CWE IDs
CWE IDs exist in both dataframes, but they are not recorded in the same way. To normalize them, I need to prepend each ID in the CWE data with `CWE-`.

In [19]:
def prepend(col, txt):
    col[:] = col.apply(lambda x: f'{txt}{x}' if pd.notna(x) else x)

prepend(cwes['cwe_id'], 'CWE-')
prepend(cwes['rel_id'], 'CWE-')

### Checking For Duplicates

In [37]:
cwes = cwes.drop_duplicates(keep='first')
print(f'There are now {cwes.duplicated().sum()} duplicates in the dataframe.')

There are now 0 duplicates in the dataframe.


## Merging the Dataframes
We should perform two types of merge: an *inner* and a *leftward* merge. The inner merge will look at the `cwe_id`, `cve_id`, and `observed_vulnerability` columns and if a given CWE ID and CVE ID exists in both datasets, it will keep *only* the data associated with these IDs across both datasets. The leftward merge will keep everything from the inner merge, but also all of the data from the CVE dataset. This will enable analyses of all the CVE descriptions and their relationship to CWEs whose tech class is `ICS/OT`, for example. 

### Inner Merge
Because of a few unexpected problems I ran into during the first merge, the following section is dedicated to the process of me working out why the data I expected to be merged together was not.

In [25]:
num_unique_cve_ids = cves['cve_id'].nunique()
non_null_cwe_ids_in_cves = cves['cwe_id'].notna().sum()
num_unique_cwes_in_cves = cves['cwe_id'].nunique()

num_unique_cwe_ids = cwes['cwe_id'].nunique()
non_null_cves_in_cwes = cwes['observed_vulnerability'].notna().sum()
num_unique_cves_in_cwes = cwes['observed_vulnerability'].nunique()

print(f"We have a CVE dataset that has {num_unique_cve_ids} unique CVE observations, {non_null_cwe_ids_in_cves} of which have CWE IDs. Of these CWE IDs, {num_unique_cwes_in_cves} are unique.")
print(f"We also have a CWE dataset with {len(cwes)} observations representing an exploded {num_unique_cwe_ids} CWE IDs. {non_null_cves_in_cwes} of these observations contain CVE IDs in their observed vulnerability attribute. Of these observed vulnerabilities, {num_unique_cves_in_cwes} are unique CVE IDs.")

We have a CVE dataset that has 246505 unique CVE observations, 58852 of which have CWE IDs. Of these CWE IDs, 664 are unique.
We also have a CWE dataset with 4157 observations representing an exploded 964 CWE IDs. 2975 of these observations contain CVE IDs in their observed vulnerability attribute. Of these observed vulnerabilities, 2043 are unique CVE IDs.


In [26]:
unique_cwe_ids = cwes['cwe_id'].unique()
print('The following observations \033[3mshould\033[0m be appended with whatever available CWE field data is available.')
cves[cves['cwe_id'].isin(unique_cwe_ids)]

The following observations should be appended with whatever available CWE field data is available.


,cve_id,cwe_id,cve_state,date_published,description,severity,severity_score,attack_vector,attack_complexity
5,CVE-1999-0006,CWE-121,PUBLISHED,1999-09-29 04:00:00+00:00,Buffer overflow in POP servers based on BSD/Qu...,CRITICAL,9.8,NETWORK,LOW
12,CVE-1999-0013,CWE-522,PUBLISHED,1999-09-29 04:00:00+00:00,Stolen credentials from SSH clients via ssh-ag...,HIGH,8.4,LOCAL,LOW
21,CVE-1999-0022,CWE-121,PUBLISHED,1999-09-29 04:00:00+00:00,Local user gains root privileges via buffer ov...,HIGH,7.8,LOCAL,LOW
28,CVE-1999-0029,CWE-121,PUBLISHED,1999-09-29 04:00:00+00:00,root privileges via buffer overflow in ordist ...,HIGH,8.4,LOCAL,LOW
35,CVE-1999-0036,CWE-645,PUBLISHED,1999-09-29 04:00:00+00:00,IRIX login program with a nonzero LOCKOUT para...,HIGH,8.4,LOCAL,LOW
...,...,...,...,...,...,...,...,...,...
260886,CVE-2024-8003,CWE-502,PUBLISHED,2024-08-20 13:31:05.130000+00:00,A vulnerability was found in Go-Tribe gotribe-...,MEDIUM,5.1,<NA>,<NA>
260887,CVE-2024-8005,CWE-798,PUBLISHED,2024-08-20 14:00:08.350000+00:00,A vulnerability was found in demozx gf_cms 1.0...,MEDIUM,6.9,<NA>,<NA>
260888,CVE-2024-8007,CWE-295,PUBLISHED,2024-08-21 13:40:25.242000+00:00,A flaw was found in the Red Hat OpenStack Plat...,HIGH,7.5,NETWORK,HIGH
260889,CVE-2024-8022,CWE-79,PUBLISHED,2024-08-20 23:31:03.646000+00:00,A vulnerability was found in Genexis Tilgin Ho...,MEDIUM,5.3,<NA>,<NA>


In [27]:
missing_cves = [cve for cve in cwes['observed_vulnerability'].unique() if cve not in cves['cve_id'].values]
print(f"Missing CVEs in CVE Data: {len(missing_cves)}. Here they are: {missing_cves}")

Missing CVEs in CVE Data: 5. Here they are: [<NA>, 'CVE-2014-8273', 'CVE-2019-1135', 'CVE-2022-30267', 'CVE-2002-216']


In [45]:
cves[['cve_id', 'cwe_id']].isnull()

,cve_id,cwe_id
0,False,True
1,False,True
2,False,True
3,False,True
4,False,True
...,...,...
260889,False,False
260890,False,False
260891,False,True
260892,False,True


In [58]:
# The actual merge
df_left2 = cves.merge(
    cwes,
    left_on=['cwe_id'],
    right_on=['cwe_id'],
    how='inner',
    validate='m:m',
    indicator=True
)

In [62]:
df_left2[df_left2['cve_id'] == 'CVE-2021-35395']

,cve_id,cwe_id,cve_state,date_published,description_x,severity,severity_score,attack_vector,attack_complexity,name,...,detection_method,detection_desc,detection_effectiveness,mitigation_phase,mitigation_desc,mitigation_effectiveness,mitigate_notes,observed_vulnerability,vulnerability_desc,_merge


In [41]:
cwes[cwes['cwe_id'] == 'CWE-121']

,cwe_id,name,description,tech_class,bg_details,rel_id,nature_of_rel,mode_of_intro_phase,mode_of_intro_desc,likelihood,...,consequence_notes,detection_method,detection_desc,detection_effectiveness,mitigation_phase,mitigation_desc,mitigation_effectiveness,mitigate_notes,observed_vulnerability,vulnerability_desc
253,CWE-121,Stack-based Buffer Overflow,A stack-based buffer overflow condition is a c...,<NA>,There are generally several security-critical ...,CWE-788,ChildOf,Implementation,<NA>,High,...,Buffer overflows generally lead to crashes. Ot...,Fuzzing,Fuzz testing (fuzzing) is a powerful technique...,High,Operation,Use automatic buffer overflow detection mechan...,Defense in Depth,<NA>,CVE-2021-35395,Stack-based buffer overflows in SFK for wifi c...
254,CWE-121,Stack-based Buffer Overflow,A stack-based buffer overflow condition is a c...,<NA>,There are generally several security-critical ...,CWE-787,ChildOf,<NA>,<NA>,High,...,Buffer overflows often can be used to execute ...,Automated Static Analysis,"Automated static analysis, commonly referred t...",High,<NA>,Use an abstraction library to abstract away ri...,<NA>,<NA>,<NA>,<NA>
255,CWE-121,Stack-based Buffer Overflow,A stack-based buffer overflow condition is a c...,<NA>,There are generally several security-critical ...,<NA>,<NA>,<NA>,<NA>,High,...,When the consequence is arbitrary code executi...,<NA>,<NA>,<NA>,<NA>,Implement and perform bounds checking on input.,<NA>,<NA>,<NA>,<NA>
256,CWE-121,Stack-based Buffer Overflow,A stack-based buffer overflow condition is a c...,<NA>,There are generally several security-critical ...,<NA>,<NA>,<NA>,<NA>,High,...,<NA>,<NA>,<NA>,<NA>,<NA>,Do not use dangerous functions such as gets. U...,<NA>,<NA>,<NA>,<NA>
257,CWE-121,Stack-based Buffer Overflow,A stack-based buffer overflow condition is a c...,<NA>,There are generally several security-critical ...,<NA>,<NA>,<NA>,<NA>,High,...,<NA>,<NA>,<NA>,<NA>,<NA>,Run or compile the software using features or ...,Defense in Depth,These techniques do not provide a complete sol...,<NA>,<NA>


In [42]:
cves[cves['cve_id'] == 'CVE-2021-35395']

,cve_id,cwe_id,cve_state,date_published,description,severity,severity_score,attack_vector,attack_complexity
178053,CVE-2021-35395,<NA>,PUBLISHED,2021-08-16 11:07:29+00:00,Realtek Jungle SDK version v2.x up to v3.4.14B...,<NA>,NaN,<NA>,<NA>


In [43]:
df_left[df_left['cve_id'] == 'CVE-2021-35395']

,cve_id,cwe_id,cve_state,date_published,description_x,severity,severity_score,attack_vector,attack_complexity,name,...,detection_method,detection_desc,detection_effectiveness,mitigation_phase,mitigation_desc,mitigation_effectiveness,mitigate_notes,observed_vulnerability,vulnerability_desc,_merge
165529,CVE-2021-35395,<NA>,PUBLISHED,2021-08-16 11:07:29+00:00,Realtek Jungle SDK version v2.x up to v3.4.14B...,<NA>,NaN,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,left_only


In [53]:
# Standardize tech classes (transform empty text to truly null values)
df['tech_class'] = df['tech_class'].replace('', pd.NA)
print('Standardized tech classes:')
df['tech_class'].unique()

Standardized tech classes:


<StringArray>
[                     <NA>,                  'ICS/OT',
                  'Mobile',         'Cloud Computing',
               'Web Based', 'Not Technology-Specific',
          'System on Chip']
Length: 7, dtype: string

cwe_id                                                               CWE-1007
name                        Insufficient Visual Distinction of Homoglyphs ...
description                 The product displays information or identifier...
tech_class                                                          Web Based
bg_details                                                                   
rel_ids                                                                  <NA>
nature_of_rels                                                           <NA>
modes_of_intro_phases                                                    <NA>
modes_of_intro_descs                                                     <NA>
likelihood                                                             Medium
scope_of_consequences                                                    <NA>
impact_of_consequences                                                   <NA>
consequence_notes                                               

In [55]:
cwes.head(20)

,cwe_id,name,description,tech_class,bg_details,rel_ids,nature_of_rels,modes_of_intro_phases,modes_of_intro_descs,likelihood,...,consequence_notes,detection_methods,detection_descs,detection_effectiveness,mitigation_phases,mitigation_descs,mitigation_effectiveness,mitigate_notes,observed_vulnerabilities,vulnerability_descs
0,CWE-1004,Sensitive Cookie Without 'HttpOnly' Flag,The product uses a cookie to store sensitive i...,Web Based,An HTTP cookie is a small piece of data attrib...,732,ChildOf,Implementation,<NA>,Medium,...,"If the HttpOnly flag is not set, then sensitiv...",Automated Static Analysis,"Automated static analysis, commonly referred t...",High,Implementation,Leverage the HttpOnly flag when setting a sens...,High,While this mitigation is effective for protect...,CVE-2022-24045,Web application for a room automation system h...
1,CWE-1004,Sensitive Cookie Without 'HttpOnly' Flag,The product uses a cookie to store sensitive i...,Web Based,An HTTP cookie is a small piece of data attrib...,<NA>,<NA>,<NA>,<NA>,Medium,...,If the cookie in question is an authentication...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,CVE-2014-3852,CMS written in Python does not include the HTT...
2,CWE-1004,Sensitive Cookie Without 'HttpOnly' Flag,The product uses a cookie to store sensitive i...,Web Based,An HTTP cookie is a small piece of data attrib...,<NA>,<NA>,<NA>,<NA>,Medium,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,CVE-2015-4138,Appliance for managing encrypted communication...
3,CWE-1007,Insufficient Visual Distinction of Homoglyphs ...,The product displays information or identifier...,Web Based,,451,ChildOf,Architecture and Design,This weakness may occur when characters from v...,Medium,...,An attacker may ultimately redirect a user to ...,Manual Dynamic Analysis,"If utilizing user accounts, attempt to submit ...",Moderate,Implementation,Use a browser that displays Punycode for IDNs ...,<NA>,<NA>,CVE-2013-7236,web forum allows impersonation of users with h...
4,CWE-1007,Insufficient Visual Distinction of Homoglyphs ...,The product displays information or identifier...,Web Based,,<NA>,<NA>,Implementation,<NA>,Medium,...,<NA>,<NA>,<NA>,<NA>,<NA>,Use an email client that has strict filters an...,<NA>,<NA>,CVE-2012-0584,Improper character restriction in URLs in web ...
5,CWE-1007,Insufficient Visual Distinction of Homoglyphs ...,The product displays information or identifier...,Web Based,,<NA>,<NA>,<NA>,<NA>,Medium,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,CVE-2009-0652,Incomplete denylist does not include homoglyph...
6,CWE-1007,Insufficient Visual Distinction of Homoglyphs ...,The product displays information or identifier...,Web Based,,<NA>,<NA>,<NA>,<NA>,Medium,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,CVE-2017-5015,web browser does not convert hyphens to punyco...
7,CWE-1007,Insufficient Visual Distinction of Homoglyphs ...,The product displays information or identifier...,Web Based,,<NA>,<NA>,<NA>,<NA>,Medium,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,CVE-2005-0233,homoglyph spoofing using punycode in URLs and ...
8,CWE-1007,Insufficient Visual Distinction of Homoglyphs ...,The product displays information or identifier...,Web Based,,<NA>,<NA>,<NA>,<NA>,Medium,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,CVE-2005-0234,homoglyph spoofing using punycode in URLs and ...
9,CWE-1007,Insufficient Visual Distinction of Homoglyphs ...,The product displays information or identifier...,Web Based,,<NA>,<NA>,<NA>,<NA>,Medium,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,CVE-2005-0235,homoglyph spoofing using punycode in URLs and ...


In [92]:
print(cves['cwe_id'].unique())
print(cwes['cwe_id'].unique())

<StringArray>
[      <NA>,  'CWE-121',  'CWE-522',  'CWE-645',   'CWE-78',  'CWE-476',
  'CWE-200',  'CWE-119',  'CWE-269',  'CWE-219',
 ...
 'CWE-1240',  'CWE-449',  'CWE-166',  'CWE-491',  'CWE-154',  'CWE-702',
  'CWE-374',  'CWE-386',  'CWE-624', 'CWE-1262']
Length: 665, dtype: string
<StringArray>
['CVE-1004', 'CVE-1007',  'CVE-102', 'CVE-1021', 'CVE-1022', 'CVE-1023',
 'CVE-1024', 'CVE-1025',  'CVE-103', 'CVE-1037',
 ...
   'CVE-94',  'CVE-940',  'CVE-941',  'CVE-942',  'CVE-943',   'CVE-95',
   'CVE-96',   'CVE-97',   'CVE-98',   'CVE-99']
Length: 964, dtype: string


In [29]:
print(f'There are {df.duplicated().sum()} duplicate rows in the dataframe.')

dups = df[['cve_id', 'cwe_id']].dropna().duplicated().sum()
print(f'There are {dups} CVEs with the same CWEs.')

There are 0 duplicate rows in the dataframe.
There are 0 CVEs with the same CWEs.


In [111]:
desc = cves[cves['cve_id'] == 'CVE-2022-37061']['description']

print(desc.iloc[0])

All FLIR AX8 thermal sensor cameras version up to and including 1.46.16 are vulnerable to Remote Command Injection. This can be exploited to inject and execute arbitrary shell commands as the root user through the id HTTP POST parameter in the res.php endpoint. A successful exploit could allow the attacker to execute arbitrary commands on the underlying operating system with the root privileges.


In [77]:
cves[cves['cwe_id'] == 'CVE-2018-13379']

,cve_id,cwe_id,cve_state,date_published,description,severity,severity_score,attack_vector,attack_complexity
